# Imports and settings

In [25]:
from openmm.app import *
from openmm import *                    
from openmm.unit import *
from openmmtools import forces
import mdtraj as md
import numpy as np
import parmed as pmd
import bz2
import os
from openmm import CustomIntegrator
from openmm.unit import kilojoules_per_mole, is_quantity
from openmm.unit import *
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt

In [3]:
# define a function for creating RMSD restraints 
def create_rmsd_restraint(positions, atom_indicies):
    rmsd_cv = RMSDForce(positions, atom_indicies)
    energy_expression = 'step(dRMSD) * (K_RMSD/2) * dRMSD^2; dRMSD = (RMSD-RMSD0);'                                                  
    energy_expression += 'K_RMSD = %f;' % spring_constant.value_in_unit_system(md_unit_system)                                       
    energy_expression += 'RMSD0 = %f;' % restraint_distance.value_in_unit_system(md_unit_system)                                     
    restraint_force = CustomCVForce(energy_expression)                                                                               
    restraint_force.addCollectiveVariable('RMSD', rmsd_cv)                                                                           
    return restraint_force                                                                                                           
                                                                  

In [13]:
# define a function to list force groups and modify system context                                                       
def forcegroupify(system):                                        
    forcegroups = {}                                              
    for i in range(system.getNumForces()):                                                                                           
        force = system.getForce(i)                           
        force.setForceGroup(i) 
        forcegroups[force] = i                                    
    return forcegroups

# Setup a quick simulation system + state + CVForces

In [6]:
sim_temp = 300.0 * kelvin
H_mass = 4.0 * amu #Might need to be tuned to 3.5 amu 
time_step = 0.002 * picosecond  
nb_cutoff = 10.0 * angstrom                                                                                                          
box_padding = 12.0 * angstrom
salt_conc = 0.15 * molar
receptor_path="./villin.pdb"
current_file="villin-solvated"
# Misc parameters                                                 
restraint_distance = 0.0 * angstroms                              
restart_freq = 10
log_freq = 1                                                                                                                         
prd_steps = 100     

In [9]:
## Load an already solvated PDB file and set up the system + state
pdb = PDBFile(receptor_path)
omm_forcefield = ForceField("amber/ff14SB.xml", "amber14/tip3p.xml")
system = omm_forcefield.createSystem(pdb.topology, 
                                         nonbondedMethod=PME, 
                                         nonbondedCutoff=nb_cutoff, 
                                         constraints=HBonds, 
                                         rigidWater=True,
                                         hydrogenMass=H_mass)
system.addForce(MonteCarloBarostat(1*bar, sim_temp))


5

## Quickly identify a couple of atoms that we care about

In [11]:
prot_top = md.Topology.from_openmm(pdb.topology)
# find two atoms and save their inds
d1_atom1_ind = prot_top.select("residue 6 and name CZ")[0]
d1_atom2_ind = prot_top.select("residue 10 and name CZ")[0] 
d2_atom2_ind = prot_top.select("residue 17 and name CZ")[0]

In [28]:
# Let's define a torsion we will use later
phe_chi = [72, 74, 76, 79]

In [15]:
# Define a way to create a basic CV force (using two distances)
def CreateCVForce(system, d1_atom1_ind, d1_atom2_ind, d2_atom2_ind):

    cvforce = CustomCVForce("0")
    system.addForce(cvforce)
    D1 = CustomBondForce("r")
    D1.addBond(d1_atom2_ind, d1_atom1_ind)
    print(D1.getNumGlobalParameters())
    #D1.addGlobalParameter("dist", "r")
    D2 = CustomBondForce("r")
    D2.addBond(d2_atom2_ind, d1_atom1_ind)
    #D2.addGlobalParameter("dist", "r")
    cvforce.addCollectiveVariable("D1", D1)
    cvforce.addCollectiveVariable("D2", D2)
    return cvforce

In [19]:
# Setup a reusable platform
platform = Platform.getPlatformByName('OpenCL')
platform.setPropertyDefaultValue('Precision', 'mixed')

# Let's setup some Accelerated Molecular Dynamics with a new Langevin Integrator

To set up an Accelerated MD (AMD) simulation in OpenMM, we could use the default AMD object.\
However this uses a Verlet integrator rather than Langevin dynamics. Luckily, OpenMM gives us the tools to write this ourselves, using the `CustomIntegrator` object!  

For Langevin-AMD, we're effectively adding a boost potential to the simulation. 
This boosted potential botential takes the form of\
$V'(r) = V_{0}(r) + \frac{(E-V(r))^2}{(\alpha+E-V_0(r))}$ \ 
where $V_0(r)$ is the initial potential energy\
and $\alpha$ and E are hyper parameters defining the potential.\
For further details on values to choose, see Hamelberg et al., J. Chem. Phys. 127, 155102 (2007).

## Let's define our custom integrator

In [20]:
kB = BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA


class LangevinAMDIntegrator(CustomIntegrator):
    """AMDIntegrator implements the aMD integration algorithm.
    The system is integrated based on a modified potential.  Whenever the energy V(r) is less than a
    cutoff value E, the following effective potential is used:
    V*(r) = V(r) + (E-V(r))^2 / (alpha+E-V(r))
    For details, see Hamelberg et al., J. Chem. Phys. 127, 155102 (2007).
    """

    def __init__(self, dt, alpha, E, temperature=300*kelvin, collision_rate=1.0/picosecond):
        """Create an AMDIntegrator.
        Parameters
        ----------
        dt : time 
            The integration time step to use (simtk quantity)
        alpha : energy
            The alpha parameter to use
        E : energy
            The energy cutoff to use
        temperature : temperature
            temperature of the system
        collision rate : collision rate
            Collision for the thermostat to update on
        """
        gamma = collision_rate*picoseconds # add this because SWIG somehow?
        CustomIntegrator.__init__(self, dt)
        # aMD boost parameters
        self.addGlobalVariable("alpha", alpha)
        self.addGlobalVariable("E", E)
        self.addPerDofVariable("oldx", 0)
        self.addGlobalVariable("deltaV", 0)
        self.addGlobalVariable("V0", 0)
        self.addPerDofVariable("sigma", 0)
        self.addUpdateContextState();
        
        # Comment out old AMD method 
        # self.addComputePerDof("v", "v+dt*fprime/m; fprime=f*((1-modify) + modify*(alpha/(alpha+E-energy))^2); modify=step(E-energy)")
        # self.addComputePerDof("oldx", "x")
        # self.addComputePerDof("x", "x+dt*v")
        # self.addConstrainPositions()
        # self.addComputePerDof("v", "(x-oldx)/dt")

        # new variables for Langevin kernel
        self.addGlobalVariable('kT', kB * temperature)
        self.addComputePerDof("sigma", "sqrt(kT/m)")
        self.addGlobalVariable("a", np.exp(-1 * gamma)) #vscale?
        self.addGlobalVariable("b", np.sqrt(1 - np.exp(-2 * gamma))) # noise-scale?

        #before position restraints
        #self.addPerDofVariable("oldx", 0)

        # Compute steps for Langevin - attempt at VRORV splitting?
        # The below dof_string lines are comments on how to modify fprime (thinking out loud basically)
        # dof_string+= "v + (dt / 2) * fprime/m;" # standard v updating for VRORV (I think)
        # dof_string+="fprime=f*((1-modify) + modify*(alpha/(alpha+E-energy))^2);" # the right hand addition is the secret amd sauce
        # dof_string = "modify=step(E-energy);" # this is defining the modify scaling
        # Put the above together into a single new -line for computing v
        self.addComputePerDof("v", "v + (dt / 2) * fprime / m; fprime=f*((1-modify) +   modify*(alpha/(alpha+E-energy))^2); modify=step(E-energy)")
        # langevin like normal
        self.addComputePerDof("x", "x + (dt / 2)*v")
        self.addComputePerDof("oldx", "x")
        self.addConstrainPositions()
        self.addComputePerDof("v", "v + (x - oldx)/(dt / 2)")
        self.addComputePerDof("v", "(a * v) + (b * sigma * gaussian)") # taken from openmm-tutorial on custom integrators (link 1 above)
        self.addComputePerDof("x", "x + (dt / 2)*v")
        self.addComputePerDof("oldx", "x")
        self.addConstrainPositions()
        self.addComputePerDof("v", "v + (x - oldx) / (dt / 2)")
        # Use the same line as what you calculated above
        self.addComputePerDof("v", "v + (dt / 2) * fprime / m; fprime=f*((1-modify) + modify*(alpha/(alpha+E-energy))^2); modify=step(E-energy)")
        # compute the V and deltaV values
        self.addComputeGlobal("V0", "energy")
        self.addComputeGlobal("deltaV","modify*(E-energy)^2/(alpha+E-energy); modify=step(E-energy)")
        self.addConstrainVelocities()




    def getAlpha(self):
        """Get the value of alpha for the integrator."""
        return self.getGlobalVariable(0)*kilojoules_per_mole

    def setAlpha(self, alpha):
        """Set the value of alpha for the integrator."""
        self.setGlobalVariable(0, alpha)

    def getE(self):
        """Get the energy threshold E for the integrator."""
        return self.getGlobalVariable(1)*kilojoules_per_mole

    def setE(self, E):
        """Set the energy threshold E for the integrator."""
        self.setGlobalVariable(1, E)

    def getEffectiveEnergy(self, energy):
        """Given the actual potential energy of the system, return the value of the effective potential."""
        alpha = self.getAlpha()
        E = self.getE()
        if not is_quantity(energy):
            energy = energy*kilojoules_per_mole # Assume kJ/mole
        if (energy > E):
            return energy
        return energy+(E-energy)*(E-energy)/(alpha+E-energy)


And now we can just define it using (Using arbitrary numbers - you should choose your own)

Briefly: To choose your own E and $\alpha$ values, the "best practice" is to run an Eq. MD simulation for ~10 nanoseconds, and look at the distribution of potential energy values.\

You'll want to set E=<V(r)> of the V(r) distribution and $\alpha$ to some fraction of E.\ 
Typically $\alpha=0.2V(r)$ is a reasonable starting point 

Now let's define the integrator object:

In [22]:
lang_amd_integrator = LangevinAMDIntegrator(dt=time_step,
                                            alpha=13060*kilojoules_per_mole,
                                            E=-63500*kilojoules_per_mole,
                                            temperature=300*kelvin,
                                            collision_rate=1.0/picosecond)

Now let's define the simulation!

In [ ]:
lang_amd_simulation = Simulation(pdb.topology, system, lang_amd_integrator, platform)

In [ ]:
# Now let's add a couple of CVForce (with Force=0) to track distances:
forcegroups = forcegroupify(system)
cvforce = CreateCVForce(system)
cvforce.setForceGroup(31)

At this point you can treat the `Simulation` object like any other

Let's say you wanted to track the boost potential applied at each frame.\
From this set of boosts you can reconstruct the effective FES explored.
 

Let's define reporters and minimize

In [ ]:
# setup reporters
lang_amd_simulation.reporters.append(DCDReporter('langevin-amd/'+current_file+''+ '.dcd', restart_freq))
lang_amd_simulation.reporters.append(CheckpointReporter('langevin-amd/'+current_file+''+ '.chk', min(prd_steps, 10*restart_freq)))
lang_amd_simulation.reporters.append(StateDataReporter(open('langevin-amd/log.' + current_file+'', 'w'), log_freq, step=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, temperature=True, volume=True, density=True, speed=True))

# minimize
lang_amd_simulation.context.setPositions(pdb.positions)
lang_amd_simulation.minimizeEnergy()

In [ ]:
# Now run the simulation, tracking the boost using the Integrator
with open("./AMD-deltaV-vals.dat", "a") as f:
    for i in range(300):
        lang_amd_simulation.step(10)
        n_steps = str(lang_amd_simulation.context.getStepCount())
        deltaV_val = str(lang_amd_integrator.getGlobalVariable(2))
        v0 = str(lang_amd_integrator.getGlobalVariable(3))
        d1, d2 = cvforce.getCollectiveVariableValues(lang_amd_simulation.context)
        f.write("%s %s %s %s %s \n" % (n_steps, v0, deltaV_val, str(d1), str(d2)))

# Langevin Gaussian Accelerated MD - a Gaussian boost instead of normal

In [24]:
# Generate a customIntegrator for Gaussian Accelerated Molecular Dynamics (GAMD)

kB = BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA

'''
Useful links in the writing of this integrator: 
Writing custom integrators: https://github.com/choderalab/openmm-tutorials/blob/master/02%20-%20Integrators%20and%20sampling.ipynb
AMD integrator: https://github.com/openmm/openmm/blob/master/wrappers/python/openmm/amd.py
GAMD from Miao lab: https://github.com/MiaoLab20/gamd-openmm
'''

class GAMDLangevinIntegrator(CustomIntegrator):
    """GAMDLangevinIntegrator implements the aMD integration algorithm.
    The system is integrated based on a modified potential.  Whenever the energy V(r) is less than a
    cutoff value E, the following effective potential is used:
    V*(r) = V(r) + ((k/2)*(E-V(r))**2)
    """

    def __init__(self, dt, E, k, temperature=300*kelvin, collision_rate=1.0/picosecond):
        """Create an GAMDLangevinIntegrator.
        Parameters
        ----------
        dt : time
            The integration time step to use
        k : energy
            The k parameter to use
        E : energy
            The energy cutoff to use
        temperature : temperature
            temperature of the system
        collision rate : collision rate
            Collision for the thermostat to update on
        """
        gamma = collision_rate*picoseconds # add this because SWIG somehow?
        CustomIntegrator.__init__(self, dt)
        # aMD boost parameters
        self.addGlobalVariable("k", k)
        self.addGlobalVariable("E", E)
        self.addPerDofVariable("oldx", 0)
        self.addGlobalVariable("deltaV", 0)
        self.addGlobalVariable("V0", 0)
        self.addPerDofVariable("sigma", 0)
        self.addUpdateContextState();
        
        # Comment out old AMD method - verlet I believe
        # self.addComputePerDof("v", "v+dt*fprime/m; fprime=f*((1-modify) + modify*(alpha/(alpha+E-energy))^2); modify=step(E-energy)")
        # self.addComputePerDof("oldx", "x")
        # self.addComputePerDof("x", "x+dt*v")
        # self.addConstrainPositions()
        # self.addComputePerDof("v", "(x-oldx)/dt")

        # new variables for Langevin kernel
        self.addGlobalVariable('kT', kB * temperature)
        self.addComputePerDof("sigma", "sqrt(kT/m)")
        self.addGlobalVariable("a", np.exp(-1 * gamma)) #vscale?
        self.addGlobalVariable("b", np.sqrt(1 - np.exp(-2 * gamma))) # noise-scale?

        #before position restraints
        #self.addPerDofVariable("oldx", 0)

        # Compute steps for Langevin - attempt at VRORV splitting?
        # Original Langevin line in AMDIntegrator - replace this with GAMD
        # self.addComputePerDof("v", "v + (dt / 2) * fprime / m; fprime=f*((1-modify) + modify*(alpha/(alpha+E-energy))^2); modify=step(E-energy)")

        # The below dof_string lines are comments on how to modify fprime (thinking out loud basically)
        # dof_string = "modify=step(E-energy);" # this is defining the scaling
        # dof_string+= "v + (dt / 2) * fprime/m;" # standard v updating for VRORV (I think)
        # dof_string+="fprime=f*((1-modify) + modify*(1-(k*E-energy)));" # the right hand addition is the secret amd sauce

        # Put the above together into a single new -line for computing v
        self.addComputePerDof("v", "v + (dt/2) * fprime/m; fprime=f*((1-modify) + modify*(1-k*(E-energy))); modify=step(E-energy)")

        # now langevin like normal
        self.addComputePerDof("x", "x + (dt / 2)*v")
        self.addComputePerDof("oldx", "x")
        self.addConstrainPositions()
        self.addComputePerDof("v", "v + (x - oldx)/(dt / 2)")
        self.addComputePerDof("v", "(a * v) + (b * sigma * gaussian)")
        self.addComputePerDof("x", "x + (dt / 2)*v")
        self.addComputePerDof("oldx", "x")
        self.addConstrainPositions()
        self.addComputePerDof("v", "v + (x - oldx) / (dt / 2)")
        # Update v with the same as the first line above
        # comment out old line from AMDIntegrator attempt
        #self.addComputePerDof("v", "v + (dt / 2) * fprime / m; fprime=f*((1-modify) + modify*(alpha/(alpha+E-energy))^2); modify=step(E-energy)")
        self.addComputePerDof("v", "v + (dt/2) * fprime/m; fprime=f*((1-modify) + modify*(1-k*(E-energy))); modify=step(E-energy)")
        # compute the V and deltaV values
        self.addComputeGlobal("V0", "energy")
        #self.addComputeGlobal("deltaV","modify*(E-energy)^2/(alpha+E-energy); modify=step(E-energy)")
        self.addComputeGlobal("deltaV","modify*((k / 2 * (E-energy)^2)); modify=step(E-energy)")
        self.addConstrainVelocities()




    def getk(self):
        """Get the value of k for the integrator."""
        return self.getGlobalVariable(0)*kilojoules_per_mole

    def setk(self, k):
        """Set the value of k for the integrator."""
        self.setGlobalVariable(0, k)

    def getE(self):
        """Get the energy threshold E for the integrator."""
        return self.getGlobalVariable(1)*kilojoules_per_mole

    def setE(self, E):
        """Set the energy threshold E for the integrator."""
        self.setGlobalVariable(1, E)

    def getEffectiveEnergy(self, energy):
        """Given the actual potential energy of the system, return the value of the effective potential."""
        k = self.getk()
        E = self.getE()
        if not is_quantity(energy):
            energy = energy*kilojoules_per_mole # Assume kJ/mole
        if (energy > E):
            return energy*kilojoules_per_mole # Assume kJ/mole
        boost = (0.5 * k * (E-energy)*(E-energy)) / kilojoules_per_mole / kilojoules_per_mole
        return energy + boost #energy+(E-energy)*(E-energy)/(alpha+E-energy)

Let's say we had some CSV file that had the potential energy distribution
How would we define the parameters that go into the GAMD Integrator? 

In [ ]:
# load in and estimate Gamd parameters from langevin simualtion
table_vals = pd.read_csv("/home/sukrit/lilac/data/sampling-test/langevin-basic/log.langevin-basic")
energies = table_vals["Potential Energy (kJ/mole)"].values

In [ ]:
#potential energy statistics:
Vmax = energies.max()
Vmin = energies.min()
Vavg = energies.mean()
Vstd = np.std(energies)
print(f'Vmax: {Vmax},\nVmin: {Vmin},\nVavg: {Vavg},\nVstd: {Vstd}')

In [ ]:
#set the desired maximum standard deviation of the boost potential to be 10kT: 
sigma_0 = (MOLAR_GAS_CONSTANT_R * sim_temp ).value_in_unit(kilojoule_per_mole) * 10
print(f'Sigma0: {sigma_0}')


In [ ]:
# define k_0 and k
k_0 = min(1, sigma_0/Vstd * ((Vmax-Vmin)/(Vmax-Vavg)))
k = k_0 * (1 / (Vmax - Vmin) )
print(f'k_0: {k_0},\nk: {k}')

And lastly - let's define the GAMD integrator object:


In [ ]:
# Define the gamd_integrator
gamd_integrator = GAMDLangevinIntegrator(dt=time_step,
                                            k=k,
                                            E=Vmax,
                                            temperature=300*kelvin,
                                            collision_rate=1.0/picosecond)

Everything else happens as with AMD for defining a `Simulation` object and running/tracking
a simulation.

# Metadynamics

## Defining our Gaussian boost potential values

Let's say I already had run that Langevin dynamics simulation - we're going to use a CSV file to \
best identify our Gaussian boosts

In [ ]:
columts = ['step', 'd1', 'd2', 'NaN']
cv_eq = pd.read_csv("./eq-vals.dat",
                    delimiter=" ",
                   names=columts)

d1_eq = cv_eq['d1'].values
d2_eq = cv_eq['d2'].values

In [ ]:
import matplotlib.pyplot as plt
plt.hist(d1_eq)
print("average: %s \n error %s" % (np.mean(d1_eq), np.std(d1_eq)))
print("error value should thus be %s" % (np.std(d1_eq) / 2))

In [ ]:
plt.hist(d2_eq)
print("average: %s \n error %s" % (np.mean(d2_eq), np.std(d2_eq)))
print("error value should thus be %s" % (np.std(d2_eq) / 2))

## Setting up our metadynamics simulation

From the code above let's say we define our boosts as well as we know them:

This time we're going to operate a MetaD simulation on 3 CVs: 2 distances and 1 dihedral.

We defined our distance and dihedral up high!

In [ ]:
# Define BiasVariables
d1 = CustomBondForce('r')
d1.addBond(d1_atom1_ind, d1_atom2_ind)
d1_bias = BiasVariable(d1, 0.3, 3.0, 0.05, periodic=False)

d2=CustomBondForce('r')
d2.addBond(d1_atom1_ind, d2_atom2_ind)
d2_bias = BiasVariable(d2, 1.2, 5.0, 0.05, periodic=False)

chi = CustomTorsionForce('theta')
chi.addTorsion(phe_chi[0], phe_chi[1], phe_chi[2], phe_chi[3])
chi_bias = BiasVariable(chi, -np.pi, np.pi, 0.35, True, 181)